In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [10]:
url = "https://www.imdb.com/search/title/?title_type=video_game&num_votes=,5000,&sort=user_rating,desc&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=87cca6a7-a16d-42d9-b9de-6aace99ec40a&pf_rd_r=ERFX36S273PQKZHMN3NF&pf_rd_s=center-6&pf_rd_t=60601&pf_rd_i=video-games&ref_=fea_vg_scg_ats_toprated_hd"

response = requests.get(url)
response.status_code

200

In [11]:
page = response.text

In [12]:
soup = BeautifulSoup(page, "lxml")

In [13]:
soup.find('div',class_='lister-item mode-advanced')

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt5667286"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt5667286/"> <img alt="Wiedzmin 3: Dziki Gon Krew i wino" class="loadlate" data-tconst="tt5667286" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMTg2ZmY0MGUtZmFjZS00YjkxLTlmMWEtMDE0ZWQwYzBlODA2XkEyXkFqcGdeQXVyMzUwNzgzNzg@._V1_UY98_CR5,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt5667286/">Wiedzmin 3: Dziki Gon Krew i wino</a>
<span class="lister-item-year text-muted unbold">(2016 Video Game)</span>
</h3>
<p class="text-muted">
<span class="genre">
Action, Adventure, Drama            </span>
</p>
<div class="ratings-bar">
<div class="inline-block ratin

In [87]:
title = soup.find('h3',class_="lister-item-header").a.text
title

'Wiedzmin 3: Dziki Gon Krew i wino'

In [88]:
year = soup.find('span',class_="lister-item-year text-muted unbold").text.replace('(','').replace(')','')
year

'2016 Video Game'

In [179]:
Gtype = soup.find('p',class_="text-muted").text.strip()
Gtype

'Action, Adventure, Drama'

In [90]:
rate = soup.find('div',class_="ratings-bar").find('strong').text
rate

'9.8'

In [123]:
descr = soup.find_all('p',class_="text-muted")[1].text.strip()
descr

"Geralt is in the southern province of Toussaint where a monstrous serial killer is targeting knights with a dark past. Geralt and his old vampire friend investigate the killer's motives."

In [158]:
Director = soup.find('p',class_='').text.split('|')[0].strip().replace('\n','')
Director

'Director:Konrad Tomaszkiewicz'

In [159]:
star = soup.find('p',class_='').text.split('|')[1].strip().replace('\n','')
star

'Stars:Doug Cockle, MyAnna Buring, Antonia Bernath, Mark Noble'

In [139]:
votes = soup.find('p',class_='sort-num_votes-visible').text.split()[1].replace(',','')
votes

'6806'

In [223]:
def clean_num(s):
    """cleans text scraped from website
    
    Parameters:
    s (String): raw text

    Returns:
    String: cleaned digit text

    """ 

    if s is None:
        return s
    return "".join(re.findall('\d*\.?\d+',s))

def get_data(block, tag, class_text):
    """fetch data from tag and its associated class
    
    Parameters:
    block (int): Book item 
    tag (String): tag for element we want to fetch
    class_text (String): class for element we want to fetch

    Returns:
    String: data text fetched

    """ 
    try:
        return block.find(tag, class_ = class_text).text.replace('(','').replace(')','')
    except AttributeError:
        return ""
    return

def get_data_a_tag(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).a.text
    except AttributeError:
        return ""
    return
def get_data_strong_tag(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).strong.text
    except AttributeError:
        return ""
    return
def get_data_descr(i, tag, class_text):    
    try:
        return i.find_all(tag, class_ = class_text)[1].text.strip()
    except AttributeError:
        return ""
    return
def get_data_Director(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).text.split('|')[0].strip().replace('\n','')
    except AttributeError:
        return ""
    return
def get_data_Star(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).text.split('|')[1].strip().replace('\n','')
    except AttributeError:
        return ""
    return
def get_data_votes(i, tag, class_text):    
    try:
        return i.find(tag, class_ = class_text).text.split()[1].replace(',','')
    except AttributeError:
        return ""
    return

In [224]:
#Test with the first videio game to ensure functions are working
for i in (soup.find('div','lister-item mode-advanced')):
    print(get_data_a_tag(i, "h3","lister-item-header"))
    print(get_data(i, "span", "lister-item-year text-muted unbold"))
    print(get_data(i, "p", "text-muted").strip())
    print(clean_num(get_data_strong_tag(i, "div","ratings-bar")))
    print(get_data_descr(i, "p","text-muted"))
    print(get_data_Director(i, "p", ""))
    print(get_data_Star(i, "p", ""))
    print(clean_num(get_data_votes(i, "p","sort-num_votes-visible")))

TypeError: find() takes no keyword arguments

In [218]:
    print(clean_num(get_data_votes(i, "p","sort-num_votes-visible")))

6806


In [229]:
Vgame_list = []

for i in (soup.find_all('div','lister-item mode-advanced')):
    title = get_data_a_tag(i, "h3","lister-item-header")
    year = get_data(i, "span", "lister-item-year text-muted unbold")
    Gtype = get_data(i, "p", "text-muted").strip()
    rate = clean_num(get_data_strong_tag(i, "div","ratings-bar"))
    descr = get_data_descr(i, "p","text-muted")
    Director = get_data_Director(i, "p", "")
    star = get_data_Star(i, "p", "")
    votes = clean_num(get_data_votes(i, "p","sort-num_votes-visible"))
    
    headers = ['title', 'year', 'game_type', 'rate', 'descr',
              'director','star','votes']
    book_dict = dict(zip(headers, [ title,
                                    year,
                                    Gtype,
                                    rate,
                                    descr, 
                                    Director,
                                    star,
                                    votes]))
    Vgame_list.append(book_dict)

    Vgame_list

IndexError: list index out of range